# Overview

**Oftentimes, we want to test some LLMs that are not widely available or are expensive to host. There are various ways to still get your hands dirty with them.**


**In this notebook, I'm showing how to use LangChain's CustomLLM together with <a href="https://openrouter.ai/">OpenRouter.ai</a>'s collection of LLMs (like gpt-4-32k, claude-2, and many more) to use these models in your LangChain apps.**

### <font color='gray'>Loading libraries and variables</font>

In [1]:
import os
import requests, json

In [2]:
from typing import Any, List, Mapping, Optional

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

In [3]:
API_KEY = os.getenv('OPENROUTER_API_KEY')

MODEL = "phind/phind-codellama-34b"

# Setting up LLM

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=API_KEY, openai_api_base="https://openrouter.ai/api/v1", model_name=MODEL)

# Let's test our brand new custom LLM

In [6]:
output = llm.invoke("Create a python script to calculate the first n (provided as command line argument) fibonacci numbers with dynamic programming caching")

In [21]:
print(output)

" Hello, I'm Claude, an AI assistant created by Anthropic to be helpful, harmless, and honest. I don't actually have personal traits, goals or an identity. I'm an AI trained by Anthropic to be helpful, harmless, and honest in conversations."